In [1]:
import os
import numpy as np
import matplotlib.pyplot as plt
from chainconsumer import ChainConsumer
import glob
import pandas as pd
import scipy.linalg
from scipy.special import erfcinv
import anesthetic
import scipy.stats

In [2]:
def loadMontePythonChain(directory):
    data = np.loadtxt(directory + '/chain_NS__accepted.txt')
    # load parameter names
    filename =  glob.glob(directory + '/*.paramnames')
    names = np.loadtxt(filename[0], dtype=str, delimiter='\t').T[0]
    # first two columns: weight & mloglkl
    params = np.concatenate((['weight','mloglkl'], names))
    # remove trailing spaces
    for idx, name in enumerate(params):
        if name[-1] == ' ':
            params[idx] = name[:-1]
    chain = pd.DataFrame(data, columns = params)
    # add the true chi2 to the chain
    chain.insert(2,'chi2',2*chain['mloglkl'])
    return(chain)

In [3]:
# chi-square values for a gaussian prior
means = np.array([0., 0.181, 1.110, 1.395, -1.265])
sigmas = np.array([1.0, 1.0, 1.0, 1.0, 1.0])
def gaussian_chi2(x, mean, sigma):
    return((x - mean)**2 / sigma**2)

In [4]:
# function that writes the chain including loglkl, prior, and posterior to a file
def add_posterior(input_folder):
    chain = loadMontePythonChain(input_folder)
    # Calculate the prior from a Gaussian chi-square 
    prior = 0.5*np.sum(gaussian_chi2(chain[['D_z1','D_z2','D_z3','D_z4','D_z5']].values, means, sigmas), axis=1)
    # add prior column to the chain
    chain.insert(3, 'prior', prior)
    # add posterior column to the chain (post = loglkl + prior) Note: This is the true log-likelihood; not the default -2*loglkl that MontePython puts in the output file!
    chain.insert(4,'post', chain['mloglkl'] + chain['prior'])
    return(chain)

In [5]:
def find_bestfit_chi2(input_folder):
    chain = add_posterior(input_folder)
    d = np.where(chain['mloglkl']<10)[0]
    chain = chain.drop(d)
    chi2 = chain['chi2'][np.where(chain['post'] == np.min(chain['post']))[0]]
    return(chi2.values[0])
def print_bestfit(input_folder):
    chain = add_posterior(input_folder)
    bf = chain.loc[np.where(chain['post'] == np.min(chain['post']))[0]]
    return(bf)

In [6]:
# K1K 
print('1cosmo: %f'%find_bestfit_chi2('../chains/K1K_1c'))
print('2cosmo: %f'%find_bestfit_chi2('../chains/K1K_bp'))

1cosmo: 152.338280
2cosmo: 150.761880


In [7]:
# K1K + clustering
print('1cosmo: %f'%find_bestfit_chi2('../chains/K1K_Cl_1c'))
print('2cosmo: %f'%find_bestfit_chi2('../chains/K1K_Cl'))

1cosmo: 157.491780
2cosmo: 156.363120


In [8]:
# K1K + clustering + Lyman-alpha
print('1cosmo: %f'%find_bestfit_chi2('../chains/K1K_ClLy_1c'))
print('2cosmo: %f'%find_bestfit_chi2('../chains/K1K_ClLy'))

1cosmo: 161.330220
2cosmo: 160.756060


In [9]:
# K1K + clustering + Lyman-alpha + CMB
print('1cosmo: %f'%find_bestfit_chi2('../chains/K1K_ClLyC_1c'))
print('2cosmo: %f'%find_bestfit_chi2('../chains/K1K_ClLyC'))

1cosmo: 161.413020
2cosmo: 160.775800


In [10]:
# K1K + Lyman-alpha
print('1cosmo: %f'%find_bestfit_chi2('../chains/K1K_Ly_1c'))
print('2cosmo: %f'%find_bestfit_chi2('../chains/K1K_Ly'))

1cosmo: 152.365620
2cosmo: 152.821400


In [11]:
# K1K + CMB
print('1cosmo: %f'%find_bestfit_chi2('../chains/K1K_CMB_1c'))
print('2cosmo: %f'%find_bestfit_chi2('../chains/K1K_CMB'))

1cosmo: 152.578820
2cosmo: 151.597700
